In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights
import matplotlib.pyplot as plt
import numpy as np
import time


In [3]:
# ======================================================================
# CUSTOM CNN on CIFAR-10 (PyTorch)
# ======================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torchvision
import torchvision.transforms as transforms

# ======================================================================
# SECTION 1: SETUP AND HYPERPARAMETERS
# ======================================================================
print("--- Setting up hyperparameters and device ---")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NUM_CLASSES = 10           # CIFAR-10 has 10 classes
IMAGE_SIZE = 64            # Resize from 32x32 → 64x64

# ======================================================================
# SECTION 2: DATA LOADING AND AUGMENTATION
# ======================================================================
print("\n--- Preparing CIFAR-10 dataset ---")

train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomCrop(IMAGE_SIZE, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

test_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ======================================================================
# SECTION 3: CUSTOM CNN MODEL DEFINITION
# ======================================================================

class CustomCNN(nn.Module):
    """Custom CNN architecture for CIFAR-10 (64x64)."""
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()
        
        # Block 1: Conv -> BN -> ReLU -> MaxPool
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Block 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Block 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# ======================================================================
# SECTION 4: TRAINING AND EVALUATION FUNCTIONS
# ======================================================================

def train_model(model, trainloader, criterion, optimizer, epochs=10, device='cpu'):
    """Train a PyTorch model."""
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        for i, (inputs, labels) in enumerate(trainloader, 0):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / total_samples
        epoch_acc = (correct_predictions / total_samples) * 100
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)
        print(f"Epoch Summary | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
    
    print('\n--- Finished Training ---')
    return history


def evaluate_model(model, testloader, criterion, device='cpu'):
    """Evaluate model performance on test set."""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    test_loss = running_loss / total_samples
    test_acc = (correct_predictions / total_samples) * 100
    print(f"\nTest Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")
    return test_loss, test_acc

# ======================================================================
# SECTION 5: TRAIN + EVALUATE PIPELINE
# ======================================================================

model = CustomCNN(num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_history = train_model(model, trainloader, criterion, optimizer,
                            epochs=EPOCHS, device=device)
evaluate_model(model, testloader, criterion, device=device)


--- Setting up hyperparameters and device ---
Using device: cuda:0

--- Preparing CIFAR-10 dataset ---
Files already downloaded and verified
Files already downloaded and verified

--- Epoch 1/20 ---
Epoch Summary | Loss: 1.7058 | Accuracy: 37.67%

--- Epoch 2/20 ---
Epoch Summary | Loss: 1.4588 | Accuracy: 46.56%

--- Epoch 3/20 ---
Epoch Summary | Loss: 1.3365 | Accuracy: 51.34%

--- Epoch 4/20 ---
Epoch Summary | Loss: 1.2391 | Accuracy: 55.37%

--- Epoch 5/20 ---
Epoch Summary | Loss: 1.1668 | Accuracy: 58.15%

--- Epoch 6/20 ---
Epoch Summary | Loss: 1.1141 | Accuracy: 60.24%

--- Epoch 7/20 ---
Epoch Summary | Loss: 1.0720 | Accuracy: 61.98%

--- Epoch 8/20 ---
Epoch Summary | Loss: 1.0389 | Accuracy: 63.40%

--- Epoch 9/20 ---
Epoch Summary | Loss: 1.0068 | Accuracy: 64.54%

--- Epoch 10/20 ---
Epoch Summary | Loss: 0.9756 | Accuracy: 65.43%

--- Epoch 11/20 ---
Epoch Summary | Loss: 0.9630 | Accuracy: 66.22%

--- Epoch 12/20 ---
Epoch Summary | Loss: 0.9340 | Accuracy: 67.42%

-

(0.7330012047767639, 75.17)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# ==============================================================================
# SECTION 1: SETUP AND HYPERPARAMETERS RESNET-18
# ==============================================================================
print("--- Setting up hyperparameters and device ---")
# Set the device to a GPU if available, otherwise use the CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
# NOTE: Training from scratch requires more epochs to converge.
# 40-50 epochs is a good starting point.
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.01 # Initial learning rate for SGD
NUM_CLASSES = 10     # For CIFAR-10


# ==============================================================================
# SECTION 2: DATA LOADING AND AUGMENTATION
# ==============================================================================
print("\n--- Preparing CIFAR-10 dataset ---")
# Define data augmentation and normalization for the training set
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Define normalization for the test set (no augmentation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create the data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# ==============================================================================
# SECTION 3: MODEL AND HELPER FUNCTIONS
# ==============================================================================

def create_resnet18_from_scratch(num_classes=10):
    """
    Creates a ResNet-18 model with random initial weights (trained from scratch).
    """
    print("Initializing ResNet-18 model with random weights.")
    # Load a ResNet18 model with weights=None for random initialization
    model = resnet18(weights=None, num_classes=num_classes)
    
    # --- MODIFICATION FOR CIFAR-10 ---
    # To improve performance on small 32x32 images, it's highly recommended
    # to make the first layer less aggressive.
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    # --- END OF MODIFICATION ---
    
    return model

def train_model(model, trainloader, criterion, optimizer, scheduler=None, epochs=20, device='cpu'):
    """Function to train a PyTorch model."""
    history = {'loss': [], 'accuracy': []}
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        if scheduler:
            scheduler.step()
        epoch_loss = running_loss / total_samples
        epoch_acc = (correct_predictions / total_samples) * 100
        history['loss'].append(epoch_loss)
        history['accuracy'].append(epoch_acc)
        print(f"Epoch Summary | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
    print('\n--- Finished Training ---')
    return history

def evaluate_model(model, testloader, criterion, device='cpu'):
    """Function to evaluate a PyTorch model's performance."""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    final_loss = running_loss / total_samples
    final_acc = (correct_predictions / total_samples) * 100
    return final_loss, final_acc


# ==============================================================================
# SECTION 4: MAIN EXECUTION BLOCK
# ==============================================================================

print("\n--- Initializing Model, Loss, and Optimizer ---")
# Create the model FROM SCRATCH and move it to the configured device
model_from_scratch = create_resnet18_from_scratch(num_classes=NUM_CLASSES)
model_from_scratch.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer to train ALL model parameters
optimizer = optim.SGD(model_from_scratch.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)

# Define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=15, gamma=0.1) # Adjusted step_size for more epochs

# --- Start Training ---
print("\n--- Training ResNet-18 from scratch ---")
history = train_model(model_from_scratch, trainloader, criterion, optimizer, scheduler=scheduler, epochs=EPOCHS, device=device)

# --- Evaluate the Model ---
print("\n--- Evaluating model on the test set ---")
final_loss, final_acc = evaluate_model(model_from_scratch, testloader, criterion, device=device)

print(f"\nFinal Test Results:")
print(f"Loss: {final_loss:.4f}")
print(f"Accuracy: {final_acc:.2f}%")

--- Setting up hyperparameters and device ---
Using device: cuda:0

--- Preparing CIFAR-10 dataset ---
Files already downloaded and verified
Files already downloaded and verified

--- Initializing Model, Loss, and Optimizer ---
Initializing ResNet-18 model with random weights.

--- Training ResNet-18 from scratch ---

--- Epoch 1/20 ---
Epoch Summary | Loss: 1.5690 | Accuracy: 41.89%

--- Epoch 2/20 ---
Epoch Summary | Loss: 1.1133 | Accuracy: 59.83%

--- Epoch 3/20 ---
Epoch Summary | Loss: 0.8755 | Accuracy: 68.89%

--- Epoch 4/20 ---
Epoch Summary | Loss: 0.7295 | Accuracy: 74.25%

--- Epoch 5/20 ---
Epoch Summary | Loss: 0.6315 | Accuracy: 78.03%

--- Epoch 6/20 ---
Epoch Summary | Loss: 0.5598 | Accuracy: 80.40%

--- Epoch 7/20 ---
Epoch Summary | Loss: 0.5104 | Accuracy: 82.42%

--- Epoch 8/20 ---
Epoch Summary | Loss: 0.4638 | Accuracy: 83.85%

--- Epoch 9/20 ---
Epoch Summary | Loss: 0.4306 | Accuracy: 84.97%

--- Epoch 10/20 ---
Epoch Summary | Loss: 0.3996 | Accuracy: 86.00%


In [4]:
# ==============================================================================
# LeNet-5 BASELINE MODEL (for CIFAR-10, from scratch, CUDA-ready)
# ==============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# ==============================================================================
# SECTION 1: DEVICE SETUP
# ==============================================================================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==============================================================================
# SECTION 2: DATASET (CIFAR-10)
# ==============================================================================
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

# ==============================================================================
# SECTION 3: LENET MODEL DEFINITION
# ==============================================================================

class LeNetCIFAR10(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNetCIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2)  # 3x32x32 → 6x32x32
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)                 # 6x16x16
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)            # 16x12x12
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# ==============================================================================
# SECTION 4: TRAINING AND EVALUATION
# ==============================================================================

def train_model(model, trainloader, criterion, optimizer, epochs=10, device='cpu'):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / total
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}]  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.2f}%")

    print("\n--- Training complete ---\n")


def evaluate_model(model, testloader, criterion, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = 100 * correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Accuracy: {accuracy:.2f}%")
    return accuracy


# ==============================================================================
# SECTION 5: MODEL INITIALIZATION AND RUN
# ==============================================================================

model = LeNetCIFAR10(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 20
train_model(model, trainloader, criterion, optimizer, epochs=EPOCHS, device=device)
evaluate_model(model, testloader, criterion, device=device)


Using device: cuda:0
Files already downloaded and verified
Files already downloaded and verified
Epoch [1/20]  Loss: 1.8204  Acc: 33.49%
Epoch [2/20]  Loss: 1.5453  Acc: 44.08%
Epoch [3/20]  Loss: 1.4516  Acc: 47.57%
Epoch [4/20]  Loss: 1.3795  Acc: 50.30%
Epoch [5/20]  Loss: 1.3394  Acc: 51.85%
Epoch [6/20]  Loss: 1.2963  Acc: 53.24%
Epoch [7/20]  Loss: 1.2628  Acc: 54.44%
Epoch [8/20]  Loss: 1.2295  Acc: 55.86%
Epoch [9/20]  Loss: 1.2013  Acc: 56.87%
Epoch [10/20]  Loss: 1.1768  Acc: 57.95%
Epoch [11/20]  Loss: 1.1523  Acc: 58.54%
Epoch [12/20]  Loss: 1.1345  Acc: 59.31%
Epoch [13/20]  Loss: 1.1144  Acc: 60.29%
Epoch [14/20]  Loss: 1.1009  Acc: 60.58%
Epoch [15/20]  Loss: 1.0804  Acc: 61.30%
Epoch [16/20]  Loss: 1.0699  Acc: 61.82%
Epoch [17/20]  Loss: 1.0501  Acc: 62.38%
Epoch [18/20]  Loss: 1.0373  Acc: 63.11%
Epoch [19/20]  Loss: 1.0274  Acc: 63.32%
Epoch [20/20]  Loss: 1.0187  Acc: 63.62%

--- Training complete ---

Test Loss: 0.9674 | Test Accuracy: 66.06%


66.06